# Output Parsers

In [4]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente engraçado e se chama {nome_assistente}'),
        ('human','{pergunta}')
    ]
)

prompt = chat_template.invoke({'nome_assistente': 'Asimo', 'pergunta': 'Qual o seu nome?'})
prompt

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente engraçado e se chama Asimo', additional_kwargs={}, response_metadata={}), HumanMessage(content='Qual o seu nome?', additional_kwargs={}, response_metadata={})])

In [6]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()
resposta = chat.invoke(prompt)
resposta

AIMessage(content='Olá! Eu sou o Asimo, seu assistente engraçado pronto para ajudar com o que precisar. E você, como posso te chamar?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 31, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5dbd6797-e729-4f97-b737-5e8256d4260b-0', usage_metadata={'input_tokens': 31, 'output_tokens': 36, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### StrOutputParser

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

output_parser.invoke(resposta)

'Olá! Eu sou o Asimo, seu assistente engraçado pronto para ajudar com o que precisar. E você, como posso te chamar?'

### Spoiler Chains

In [8]:
chain = chat_template | chat | output_parser

chain.invoke({'nome_assistente': 'Asimo', 'pergunta':'Qual o seu nome?'})

'Olá! Meu nome é Asimo, o assistente engraçado! Como posso te ajudar hoje?'

# Using .with_structured_output()

In [9]:
from typing import Optional
from pydantic import BaseModel, Field

class Piada(BaseModel):
    """Piada para contar ao usuário"""
    introducao: str = Field(description='A introdução da piada')
    punchline: str = Field(description='A conclusão da piada')
    avaliacao: Optional[int] = Field(description='O quão engraçada é a piada de 1 a 10')

llm_estruturada = chat.with_structured_output(Piada)
resposta = llm_estruturada.invoke('Conte uma piada sobre gatinhos')
resposta

Piada(introducao='Por que os gatinhos são tão bons em matemática?', punchline='Porque eles têm nove vidas para praticar!', avaliacao=8)

In [10]:
resposta.introducao

'Por que os gatinhos são tão bons em matemática?'

# Example

Digamos que temos a seguinte review de um produto:

> "Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."

E eu quero que o modelo de linguagem processe esta review para estruturá-la no seguinte formato:

```json
{
  "presente": true,
  "dias_entrega": 2,
  "percepcao_de_valor": ["um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado"]
}

```

In [11]:
review_cliente = """Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. 
Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. 
Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs alternadas para limpar as folhas do nosso gramado. 
É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."""

In [13]:
from pydantic import BaseModel, Field

class AvaliacaoReview(BaseModel):
    """Avalia review do cliente"""
    presente: bool = Field(description='True se foi para presente e False se não foi')
    dias_entrega: int = Field(description='Quantos dias para entrega do produto')
    percepcao_valor: list[str] = Field(description='Extraia qualquer frase sobre o valor ou \
    preço do produto. Retorne uma lista.')

llm_estruturada = chat.with_structured_output(AvaliacaoReview)
resposta = llm_estruturada.invoke(review_cliente)
resposta

AvaliacaoReview(presente=True, dias_entrega=2, percepcao_valor=['um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras'])